**Classify speakers using Fast Fourier Transform (FFT) and a 1D Convnet.**

Create a model to classify speakers from the frequency domain representation of speech recordings, obtained via Fast Fourier Transform (FFT).


# Get all path file

First, we mout to drive to collect data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Import**

In [ ]:
import os
import wave, os, glob
import itertools
import tensorflow as tf
import numpy as np
from tensorflow import keras

Install data for training and evaluation from gg drive

input data includes over 37210 file wavs were recorded from 61 student in class. 

Each wav file is a recording of each person's name pronunciation

In [ ]:
path_data_training = '/content/drive/MyDrive/51800875_51703107__51900348/Training'
path_data_Evaluation = '/content/drive/MyDrive/51800875_51703107__51900348/Evaluation'

# function: Create a list includes file path
# input: folder path (tranining or Evaluatioin), true for Tranining or false for Evaluation
# output: list includes file path 
def file_name_path(folder_path, bool): 
    folder_name = ''
    if(bool):
        folder_name = 'Training'
    else:
        folder_name = 'Evaluation'
    def readFileName(path):
        zero = []
        for filename in glob.glob(os.path.join(path, '*.wav')):
            zero.append(filename)
        return zero
    sub_folders_name = [name for name in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, name))]
    list1 = []
    for name in sub_folders_name:
      list1.append(readFileName(f'/content/drive/MyDrive/51800875_51703107__51900348/{folder_name}/{name}'))
    return list(itertools.chain(*list1))

In [ ]:
print('list_file_name+-_training: ', file_name_path(path_data_training, True))
print(len(file_name_path(path_data_training, True)))

list_file_name+-_training:  ['/content/drive/MyDrive/51800875_51703107__51900348/Training/NguyenQuangKhai/NguyenQuangKhai_DoanNguyenMinhPhuong_6.wav', '/content/drive/MyDrive/51800875_51703107__51900348/Training/NguyenQuangKhai/NguyenQuangKhai_LeMinhPhat_6.wav', '/content/drive/MyDrive/51800875_51703107__51900348/Training/NguyenQuangKhai/NguyenQuangKhai_LeTienDat_4.wav', '/content/drive/MyDrive/51800875_51703107__51900348/Training/NguyenQuangKhai/NguyenQuangKhai_DangThanhDat_2.wav', '/content/drive/MyDrive/51800875_51703107__51900348/Training/NguyenQuangKhai/NguyenQuangKhai_LeThanhDat_4.wav', '/content/drive/MyDrive/51800875_51703107__51900348/Training/NguyenQuangKhai/NguyenQuangKhai_HoangNgocAnhTrung_4.wav', '/content/drive/MyDrive/51800875_51703107__51900348/Training/NguyenQuangKhai/NguyenQuangKhai_DangHuuQuangTri_3.wav', '/content/drive/MyDrive/51800875_51703107__51900348/Training/NguyenQuangKhai/NguyenQuangKhai_DoanNguyenMinhPhuong_7.wav', '/content/drive/MyDrive/51800875_51703107_

In [ ]:
print('list_file_name_evaluation: ', file_name_path(path_data_Evaluation, False))
print(len(file_name_path(path_data_Evaluation, False)))

list_file_name_evaluation:  ['/content/drive/MyDrive/51800875_51703107__51900348/Evaluation/NguyenQuangKhai/NguyenQuangKhai_NguyenSonDinh_8.wav', '/content/drive/MyDrive/51800875_51703107__51900348/Evaluation/NguyenQuangKhai/NguyenQuangKhai_NguyenHoaiBao_8.wav', '/content/drive/MyDrive/51800875_51703107__51900348/Evaluation/NguyenQuangKhai/NguyenQuangKhai_VoQuocSon_8.wav', '/content/drive/MyDrive/51800875_51703107__51900348/Evaluation/NguyenQuangKhai/NguyenQuangKhai_NguyenHungHoaiNam_8.wav', '/content/drive/MyDrive/51800875_51703107__51900348/Evaluation/NguyenQuangKhai/NguyenQuangKhai_HoangTuanAnh_9.wav', '/content/drive/MyDrive/51800875_51703107__51900348/Evaluation/NguyenQuangKhai/NguyenQuangKhai_VoDangPhuc_9.wav', '/content/drive/MyDrive/51800875_51703107__51900348/Evaluation/NguyenQuangKhai/NguyenQuangKhai_HoangNgocAnhTrung_9.wav', '/content/drive/MyDrive/51800875_51703107__51900348/Evaluation/NguyenQuangKhai/NguyenQuangKhai_VoDangPhuc_8.wav', '/content/drive/MyDrive/51800875_51703

# Create labels

In [ ]:
list_file_name_training = file_name_path(path_data_training, True)
list_file_name_evaluation = file_name_path(path_data_Evaluation, False)

In [ ]:
# function: Create a list includes label
# input: list file name
# output: label 
def create_label(list_path):
    list_name = [] 
    for name in list_path:
        list_name.append(name.split('/')[-1].split('_')[1])
    return list_name

In [ ]:
train_label = create_label(list_file_name_training)
print('training label: ', train_label)

training label:  ['DoanNguyenMinhPhuong', 'LeMinhPhat', 'LeTienDat', 'DangThanhDat', 'LeThanhDat', 'HoangNgocAnhTrung', 'DangHuuQuangTri', 'DoanNguyenMinhPhuong', 'LeTienDat', 'LeMinhPhat', 'DoanNguyenMinhPhuong', 'HoVanVu', 'DoanVanPhuong', 'DinhTienAnh', 'LeThanhDat', 'DoanNguyenMinhPhuong', 'LeDangDuc', 'HoangTuanAnh', 'HoangNgocAnhTrung', 'HoangTuanAnh', 'LeDangDuc', 'DangHuuQuangTri', 'DangAnhTu', 'HoMinhHuy', 'LacMinhLong', 'DinhTienAnh', 'DinhDongThuc', 'HoVanVu', 'DangThanhDat', 'HoVanVu', 'LeTienDat', 'DinhTienAnh', 'HoVanVu', 'DinhDongThuc', 'DoanVanPhuong', 'HoangNgocAnhTrung', 'DoanVanPhuong', 'HoMinhHuy', 'LeDangDuc', 'DoanNguyenMinhPhuong', 'DinhDongThuc', 'DoanVanPhuong', 'DangHuuQuangTri', 'DinhDongThuc', 'DoNgocKhai', 'LacMinhLong', 'LeDangDuc', 'DinhDongThuc', 'HoangTuanAnh', 'HoVanVu', 'DangThanhDat', 'DoNgocKhai', 'DinhTienAnh', 'HoMinhHuy', 'HoMinhHuy', 'DoNgocKhai', 'HoMinhHuy', 'DangHuuQuangTri', 'LeThanhDat', 'DoanVanPhuong', 'LeMinhPhat', 'HoVanVu', 'DangHuuQua

In [ ]:
eval_label = create_label(list_file_name_evaluation)
print('evaluation label: ', eval_label)

evaluation label:  ['NguyenSonDinh', 'NguyenHoaiBao', 'VoQuocSon', 'NguyenHungHoaiNam', 'HoangTuanAnh', 'VoDangPhuc', 'HoangNgocAnhTrung', 'VoDangPhuc', 'NguyenTuanAnh', 'PhamTommy', 'MaiVinhHien', 'NguyenHuuThinh', 'PhamAnhThu', 'NguyenTanHoangPhuc', 'DinhTienAnh', 'DangHuuQuangTri', 'TruongCongTien', 'MaiNguyenThaiHoc', 'DangThanhDat', 'HoVanVu', 'TranDucHung', 'LeThanhDat', 'DangAnhTu', 'NguyenQuocSon', 'NguyenThanhDiNien', 'NguyenHuuHuy', 'NguyenPhuKhanh', 'NguyenVanHoang', 'LuuQuangThang', 'VoNgocTram', 'HoMinhHuy', 'TangHienPhan', 'LacMinhLong', 'VoQuocSon', 'TranHoangHieu', 'TranHoangHieu', 'TranQuangVinh', 'HoangTuanAnh', 'NguyenSonDinh', 'PhamVuLongKhai', 'LeDangDuc', 'NguyenHuuThinh', 'MaiVinhHien', 'LeTrongPhuc', 'NguyenMinhKhanh', 'NguyenHoangLong', 'NguyenHoaiBao', 'VuTrungHau', 'NguyenVanAnh', 'DoanVanPhuong', 'NguyenLiuTienTai', 'PhamVuLongKhai', 'LeDangDuc', 'MaiNguyenThaiHoc', 'NguyenVanAnh', 'NguyenTuanAnh', 'DoNgocKhai', 'XuanKimLong', 'HoangNgocAnhTrung', 'LeMinhPha

In [ ]:
# LabelEncoder
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder().fit(train_label)
labels_training = label_encoder.transform(train_label)
labels_evaluation = label_encoder.transform(eval_label)

print('labels_training', len(labels_training))
print('labels_evaluation', len(labels_evaluation))

# Dataset generation

In [ ]:
SAMPLING_RATE = 16000
BATCH_SIZE = 64
SHUFFLE_SEED = 43
EPOCHS = 50

In [ ]:
def paths_and_labels_to_dataset(audio_paths, labels):
    """Constructs a dataset of audios and labels."""
    path_ds = tf.data.Dataset.from_tensor_slices(audio_paths)
    audio_ds = path_ds.map(lambda x: path_to_audio(x))
    print(audio_ds)
    label_ds = tf.data.Dataset.from_tensor_slices(labels)
    return tf.data.Dataset.zip((audio_ds, label_ds))


def path_to_audio(path):
    """Reads and decodes an audio file."""
    audio = tf.io.read_file(path)
    audio, _ = tf.audio.decode_wav(audio, 1, SAMPLING_RATE)
    return audio

def audio_to_fft(audio):
    # Since tf.signal.fft applies FFT on the innermost dimension,
    # we need to squeeze the dimensions and then expand them again
    # after FFT
    audio = tf.squeeze(audio, axis=-1)
    fft = tf.signal.fft(
        tf.cast(tf.complex(real=audio, imag=tf.zeros_like(audio)), tf.complex64)
    )
    fft = tf.expand_dims(fft, axis=-1)

    # Return the absolute value of the first half of the FFT
    # which represents the positive frequencies
    return tf.math.abs(fft[:, : (audio.shape[1] // 2), :])

## Prepare training & validation sets


In [ ]:
train_ds = paths_and_labels_to_dataset(list_file_name_training, labels_training)
train_ds = train_ds.shuffle(buffer_size=BATCH_SIZE * 8, seed=SHUFFLE_SEED).batch(
    BATCH_SIZE
)

valid_ds = paths_and_labels_to_dataset(list_file_name_evaluation, labels_evaluation)
valid_ds = valid_ds.shuffle(buffer_size=BATCH_SIZE * 8, seed=SHUFFLE_SEED).batch(BATCH_SIZE)

# Transform audio wave to the frequency domain using `audio_to_fft`
train_ds = train_ds.map(
    lambda x, y: (audio_to_fft(x), y), num_parallel_calls=tf.data.AUTOTUNE
)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

valid_ds = valid_ds.map(
    lambda x, y: (audio_to_fft(x), y), num_parallel_calls=tf.data.AUTOTUNE
)
valid_ds = valid_ds.prefetch(tf.data.AUTOTUNE)

<MapDataset element_spec=TensorSpec(shape=(16000, 1), dtype=tf.float32, name=None)>
<MapDataset element_spec=TensorSpec(shape=(16000, 1), dtype=tf.float32, name=None)>


# Model Definition

## Build the model

Using 1D convolution layer: * keras.layers.Conv1D*

This layer creates a convolution kernel that is convolved with the layer input over a single spatial (or temporal) dimension to produce a tensor of outputs

Compile the model using Adam's default learning rate and Optimizer that implements the Adam algorithm.



In [ ]:
def residual_block(x, filters, conv_num=3, activation="relu"):
    # Shortcut
    s = keras.layers.Conv1D(filters, 1, padding="same")(x)
    for i in range(conv_num - 1):
        x = keras.layers.Conv1D(filters, 3, padding="same")(x)
        x = keras.layers.Activation(activation)(x)
    x = keras.layers.Conv1D(filters, 3, padding="same")(x)
    x = keras.layers.Add()([x, s])
    x = keras.layers.Activation(activation)(x)
    return keras.layers.MaxPool1D(pool_size=2, strides=2)(x)


def build_model(input_shape, num_classes):
    inputs = keras.layers.Input(shape=input_shape, name="input")

    x = residual_block(inputs, 16, 1)
    x = residual_block(x, 32, 1)
    x = residual_block(x, 64, 2)
    x = residual_block(x, 128, 3)
    x = residual_block(x, 128, 3)

    x = keras.layers.AveragePooling1D(pool_size=3, strides=3)(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(256, activation="relu")(x)
    x = keras.layers.Dense(128, activation="relu")(x)

    outputs = keras.layers.Dense(num_classes, activation="softmax", name="output")(x)

    return keras.models.Model(inputs=inputs, outputs=outputs)


model = build_model((SAMPLING_RATE // 2, 1), len(set(labels_training)))

model.summary()

# Compile the model using Adam's default learning rate
model.compile(
    optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# Add callbacks:
# 'EarlyStopping' to stop training when the model is not enhancing anymore
# 'ModelCheckPoint' to always keep the model that has the best val_accuracy
model_save_filename = "model.h5"

earlystopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
mdlcheckpoint_cb = keras.callbacks.ModelCheckpoint(
    model_save_filename, monitor="val_accuracy", save_best_only=True
)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 8000, 1)]    0           []                               
                                                                                                  
 conv1d_1 (Conv1D)              (None, 8000, 16)     64          ['input[0][0]']                  
                                                                                                  
 conv1d (Conv1D)                (None, 8000, 16)     32          ['input[0][0]']                  
                                                                                                  
 add (Add)                      (None, 8000, 16)     0           ['conv1d_1[0][0]',               
                                                                  'conv1d[0][0]']             

# Training

In [ ]:
tf.debugging.set_log_device_placement(True)
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=valid_ds,
    callbacks=[earlystopping_cb, mdlcheckpoint_cb],
)

Epoch 1/50
15/15 [==============================] - 100s 4s/step - loss: 5.8898 - accuracy: 0.0250 - val_loss: 4.0460 - val_accuracy: 0.0625
Epoch 2/50
15/15 [==============================] - 14s 829ms/step - loss: 3.8026 - accuracy: 0.1063 - val_loss: 3.2123 - val_accuracy: 0.2417
Epoch 3/50
15/15 [==============================] - 13s 781ms/step - loss: 2.5515 - accuracy: 0.3531 - val_loss: 1.8733 - val_accuracy: 0.5750
Epoch 4/50
15/15 [==============================] - 14s 820ms/step - loss: 1.4948 - accuracy: 0.6333 - val_loss: 1.7311 - val_accuracy: 0.5667
Epoch 5/50
15/15 [==============================] - 13s 807ms/step - loss: 1.0331 - accuracy: 0.7365 - val_loss: 1.4021 - val_accuracy: 0.6750
Epoch 6/50
15/15 [==============================] - 12s 761ms/step - loss: 0.6000 - accuracy: 0.8365 - val_loss: 1.0779 - val_accuracy: 0.7542
Epoch 7/50
15/15 [==============================] - 16s 1000ms/step - loss: 0.4365 - accuracy: 0.8781 - val_loss: 1.2666 - val_accuracy: 0.6833


# Evaluation

In [ ]:
print(model.evaluate(valid_ds))

4/4 [==============================] - 3s 410ms/step - loss: 0.8105 - accuracy: 0.8625
[0.8105051517486572, 0.862500011920929]
